http://blog.tryolabs.com/2015/02/17/python-elasticsearch-first-steps/

In [2]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "Wasp",
  "cluster_name" : "elasticsearch_huangming-yi",
  "version" : {
    "number" : "2.1.1",
    "build_hash" : "40e2c53a6b6c2972b3d13846e450e66f4375bd71",
    "build_timestamp" : "2015-12-15T13:05:55Z",
    "build_snapshot" : false,
    "lucene_version" : "5.3.1"
  },
  "tagline" : "You Know, for Search"
}



In [3]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
#let's iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
print(i)

18


In [5]:
es.get(index='sw', doc_type='people', id=5)

{u'_id': u'5',
 u'_index': u'sw',
 u'_source': {u'birth_year': u'19BBY',
  u'created': u'2014-12-10T15:20:09.791000Z',
  u'edited': u'2014-12-20T21:17:50.315000Z',
  u'eye_color': u'brown',
  u'films': [u'http://swapi.co/api/films/6/',
   u'http://swapi.co/api/films/3/',
   u'http://swapi.co/api/films/2/',
   u'http://swapi.co/api/films/1/',
   u'http://swapi.co/api/films/7/'],
  u'gender': u'female',
  u'hair_color': u'brown',
  u'height': u'150',
  u'homeworld': u'http://swapi.co/api/planets/2/',
  u'mass': u'49',
  u'name': u'Leia Organa',
  u'skin_color': u'light',
  u'species': [u'http://swapi.co/api/species/1/'],
  u'starships': [],
  u'url': u'http://swapi.co/api/people/5/',
  u'vehicles': [u'http://swapi.co/api/vehicles/30/']},
 u'_type': u'people',
 u'_version': 1,
 u'found': True}

新增 node 
```
elasticsearch -Des.node.name=Node-2
```

In [11]:
r = requests.get('http://localhost:9201')
i = 18
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i,     body=json.loads(r.content))
    i=i+1

### name 是 Darth Vader

In [12]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'4',
    u'_index': u'sw',
    u'_score': 2.7218676,
    u'_source': {u'birth_year': u'41.9BBY',
     u'created': u'2014-12-10T15:18:20.704000Z',
     u'edited': u'2014-12-20T21:17:50.313000Z',
     u'eye_color': u'yellow',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'male',
     u'hair_color': u'none',
     u'height': u'202',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'136',
     u'name': u'Darth Vader',
     u'skin_color': u'white',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/13/'],
     u'url': u'http://swapi.co/api/people/4/',
     u'vehicles': []},
    u'_type': u'people'},
   {u'_id': u'44',
    u'_index': u'sw',
    u'_score': 0.62182844,
    u'_source': {u'birth_ye

### **name 前綴詞有 lu**

In [15]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'sw',
    u'_score': 1.0,
    u'_source': {u'birth_year': u'19BBY',
     u'created': u'2014-12-09T13:50:51.644000Z',
     u'edited': u'2014-12-20T21:17:56.891000Z',
     u'eye_color': u'blue',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/',
      u'http://swapi.co/api/films/7/'],
     u'gender': u'male',
     u'hair_color': u'blond',
     u'height': u'172',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'77',
     u'name': u'Luke Skywalker',
     u'skin_color': u'fair',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/12/',
      u'http://swapi.co/api/starships/22/'],
     u'url': u'http://swapi.co/api/people/1/',
     u'vehicles': [u'http://swapi.co/api/vehicles/14/',
      u'http: